## This Jupyter Notebook follows on from the PDF Worker Notebook if you have not used that to create a series of descriptions for images within a saved folder please go back and do that first, this will not work without it!

This Jupyter notebook goes over the process of retrieving the descriptions generated in the PDF Worker Notebook and displaying the image most relevant to the user query based on these descriptions. We start again by importing some modules and defining some variables.

In [ ]:
import json
import os
import requests
from PIL import Image

Make sure to switch your description_directory and image_directory to the ones on your computer

In [ ]:
document_name = ""

api_key = ""

description_directory = r""
image_directory = r""

We then open the descriptions json file we created and store it as a string.

In [ ]:
file_name = f"{document_name}_descriptions.json"
file_path = os.path.join(description_directory, file_name)

with open(file_path, "r") as file:
    descriptions = json.load(file)

We can then create a general user query. We place this query inside a prompt and ask the gpt-4o to assess which of the descriptions is closest to the query and based on this to return an indexed number starting from 1, we also ask that if non of them are relevent to state the word "false". We can then process this output such that the image that had the description closest to the user query is shown or if no images are relevent we can say as such.

In [ ]:
query = "what is the double slit experiment"
prompt = f"out of this list {descriptions} which relates most to {query},if none of them relate to the query state only the word false otherwise state only a single number starting with the first being 0 as this is being used for indexing in python, "

message = {
            "model": "gpt-4o",
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt
                        },
                    ]
                }
            ],
            "max_tokens": 300
        }

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=message)

index = response.json()['choices'][0]['message']['content']
print(index)

filenames = []

for names in os.listdir(image_directory):
    filenames.append(names)

if index == "false" or index == "False":
     print("No image returnable")
else:
    image = Image.open(os.path.join(image_directory, filenames[int(index)]))
    image.show()
    print(descriptions[int(index)])

Hopefully you should have been able to retrieve an image of a double slit interference pattern (assuming you kept the same initial query). It can be inconsistent even at the best of times, 9/10 times the system works however sometimes the model will retireve the wrong index compared to what it meant and that ruins the whole thing. You can imagine however that given a query we can retireve a relevent image. It should be noted that although through the PDF Worker and Image Picker example we curated the images from a pdf and the descriptions were generated from a chat bot we could have also simply placed some images in the relevant folder and created a .json file with a list of our own manually written descriptions about those images in the order that they are retrieved (order is important as the index retrieved from the descriptions is the same one as for retieving the images).